# Airbyte Hubspot (已弃用)

注意：`AirbyteHubspotLoader` 已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于将数据从 API、数据库和文件集成到仓库和数据湖的 ELT 管道中。它拥有最大的 ELT 连接器到数据仓库和数据库的目录。

此加载器将 Hubspot 连接器作为一个文档加载器公开，允许您将各种 Hubspot 对象加载为文档。

## 安装

首先，您需要安装 `airbyte-source-hubspot` Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-hubspot

## 示例

请查看 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/hubspot/) 以了解有关如何配置读取器的详细信息。

配置对象应遵循的 JSON 架构可在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-hubspot/source_hubspot/spec.yaml](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-hubspot/source_hubspot/spec.yaml)。

总体结构如下所示：
```python
{
  "start_date": "<用于开始检索记录的日期，ISO 格式，例如 2020-10-20T00:00:00Z>",
  "credentials": {
    "credentials_title": "Private App Credentials",
    "access_token": "<您的私有应用程序的访问令牌>"
  }
}
```

默认情况下，所有字段都存储为文档中的元数据，文本设置为一个空字符串。通过转换读取器返回的文档来构建文档的文本。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteHubspotLoader

config = {
    # your hubspot configuration
}

loader = AirbyteHubspotLoader(
    config=config, stream_name="products"
)  # check the documentation linked above for a list of all streams

现在您可以像平常一样加载文档了

In [ ]:
docs = loader.load()

由于 `load` 方法返回一个列表，它将一直阻塞直到所有文档加载完成。为了更好地控制此过程，您还可以使用 `lazy_load` 方法，该方法返回一个迭代器而不是列表：

In [ ]:
docs_iterator = loader.lazy_load()

请记住，默认情况下页面内容为空，metadata 对象包含记录中的所有信息。要处理文档，请创建一个继承自基类的加载器，并自行实现 `_handle_records` 方法：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteHubspotLoader(
    config=config, record_handler=handle_record, stream_name="products"
)
docs = loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，而不会再次加载它们。这对于数据量大且更新频繁的源非常有用。

为了利用此功能，应存储 `last_state` 属性，并在重新创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteHubspotLoader(
    config=config, stream_name="products", state=last_state
)

new_docs = incremental_loader.load()